In [14]:
import torch
import torch.nn as nn
# 一、参数访问
"""
当通过Sequential类定义模型时,可以通过索引来访问模型的任意层。
这就像模型是一个列表一样,每层的参数都在其属性中
"""
net = nn.Sequential(
    nn.Linear(4,8),
    nn.ReLU(),
    nn.Linear(8,1))
print(net[2].state_dict())
# 每个参数都表示为参数类的一个实例。要对参数执行任何操作,首先需要访问底层的数值
print(type(net[2].bias)) # print出的class 'torch.nn.parameter.Parameter'就是一个参数类
print(net[2].bias) # net[2].bias 就是一个参数类实例 参数是复合的对象,包含值、梯度和额外信息
print(net[2].bias.data) # 通过参数类实例访问参数值
# 一次性访问所有参数
print(*[(name, param.shape) for name, param in net[0].named_parameters()]) # 访问第一个全连接层的参数
print(*[(name, param.shape) for name, param in net.named_parameters()]) # 访问所有层的参数
net.state_dict()['2.bias'].data # 通过state_dict传入 层号.参数名 访问网络参数

def block1():
    return nn.Sequential(
        nn.Linear(4,8),
        nn.ReLU(),
        nn.Linear(8,4),
        nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4): # 此处进行块的嵌套
        net.add_module(f'block {i}',block1())
    return net
rgnet = nn.Sequential(
    block2(), 
    nn.Linear(4,1))
X = torch.rand(size=(2, 4))
print(f"rgnet(X): {rgnet(X)}")
print(rgnet) # 打印rgnet的结构
print(f"第一个块的第二个子块的第一层的偏置:{rgnet[0][1][0].bias.data}") # 由于层是分层嵌套的,也可以向通过嵌套列表索引一样访问

# 二、参数初始化
"""
默认情况下,PyTorch会根据一个范围均匀地初始化权重和偏置矩阵,这个范围是根据输入和输出维度计算出的。
PyTorch的nn.init模块提供了多种预置初始化方法。
"""
def init_normal(m): # 调用pytorch的内置初始化
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)

def init_const(m): # 将参数初始化为给定的常数
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)
net.apply(init_normal)
print(f"init_normal后{net[0].weight.data[0]}, {net[0].bias.data[0]}")
net.apply(init_const)
print(f"init_const后{net[0].weight.data[0]}, {net[0].bias.data[0]}")

OrderedDict([('weight', tensor([[ 0.2106, -0.3097, -0.0554,  0.1624,  0.3370,  0.0260,  0.0954, -0.0193]])), ('bias', tensor([0.0957]))])
<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([0.0957], requires_grad=True)
tensor([0.0957])
('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))
rgnet(X): tensor([[0.0078],
        [0.0078]], grad_fn=<AddmmBackward0>)
Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)